# 비수기 linear regression

In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import statsmodels.api as sm

In [111]:
data = pd.read_csv(r"data45910.csv", encoding='cp949')

In [112]:
data['모기 개체수'] = data['모기 개체수'].str.replace(',', '').astype(int)

In [113]:
features = [
    '평균기온', '최저기온','일강수량(mm)', '평균 풍속(m/s)',
     '최소 상대습도(%)','평균 상대습도(%)', '평균 현지기압(hPa)', '평균 해면기압(hPa)',
    '합계 일조시간(hr)', '평균 전운량(1/10)', '1일전 모기 개체수','2일전 모기 개체수','3일전 모기 개체수']

target = '모기 개체수'

In [114]:
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [115]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [116]:
X_train_scaled = sm.add_constant(X_train_scaled)
X_test_scaled = sm.add_constant(X_test_scaled)

In [117]:
model = sm.OLS(y_train, X_train_scaled)
results = model.fit()

In [118]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 모기 개체수   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     87.71
Date:                Tue, 28 May 2024   Prob (F-statistic):          4.68e-131
Time:                        21:19:01   Log-Likelihood:                -5317.9
No. Observations:                 635   AIC:                         1.066e+04
Df Residuals:                     621   BIC:                         1.073e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        397.8225    818.378      0.486      0.6

In [119]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [120]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [121]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 1100709.5041
Train RMSE : 1049.1470
Train R^2 : 0.6474
Performance for TEST--------
Test MSE : 2140568.5378
Test RMSE : 1463.0682
Test R^2 : 0.5591


# 비수기 Ridge

In [122]:
from sklearn.linear_model import Ridge

In [123]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42)  # 0.25 x 0.8 = 0.2

In [124]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [125]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

alphas = [0.01, 0.1, 1, 10, 100, 1000]
train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    # Ridge 회귀 모델 생성 및 학습
    model = Ridge(alpha=alpha)
    model.fit(X_train_scaled, y_train)

    # 훈련 세트와 검증 세트에서의 예측 수행
    y_train_hat = model.predict(X_train_scaled)  
    y_val_hat = model.predict(X_val_scaled)  

    # R^2 값 계산
    train_r2 = r2_score(y_train, y_train_hat)
    val_r2 = r2_score(y_val, y_val_hat)
    
    # 결과 저장
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    # 결과 출력
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print('---------------------')


Alpha: 0.01
Train R^2: 0.7210
Validation R^2: 0.7313
---------------------
Alpha: 0.1
Train R^2: 0.7208
Validation R^2: 0.7354
---------------------
Alpha: 1
Train R^2: 0.7120
Validation R^2: 0.7639
---------------------
Alpha: 10
Train R^2: 0.5879
Validation R^2: 0.7529
---------------------
Alpha: 100
Train R^2: 0.2350
Validation R^2: 0.3368
---------------------
Alpha: 1000
Train R^2: 0.0366
Validation R^2: 0.0522
---------------------


In [126]:
model = Ridge(alpha=1) #1 채택
model.fit(X_train_scaled, y_train)

Ridge(alpha=1)

In [127]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [128]:
coefficients = pd.DataFrame({
    'Feature': features,
    'Ridge Coefficient': model.coef_
})
print(coefficients)

         Feature  Ridge Coefficient
0           평균기온        1153.594901
1           최저기온        1062.984077
2       일강수량(mm)          42.163853
3     평균 풍속(m/s)        -260.396502
4     최소 상대습도(%)       -1020.469562
5     평균 상대습도(%)         198.604711
6   평균 현지기압(hPa)         -63.356571
7   평균 해면기압(hPa)         -75.895295
8    합계 일조시간(hr)        -483.006214
9   평균 전운량(1/10)        -219.799250
10    1일전 모기 개체수        7389.928338
11    2일전 모기 개체수        2321.863652
12    3일전 모기 개체수        3180.455900


In [129]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 957224.0315
Train RMSE : 978.3783
Train R^2 : 0.7120
Performance for TEST--------
Test MSE : 3062245.8854
Test RMSE : 1749.9274
Test R^2 : 0.4375


# 비수기 Lasso

In [130]:
from sklearn.linear_model import Lasso

In [131]:
alphas = [0.01, 0.1, 1, 10, 100]
train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    # Ridge 회귀 모델 생성 및 학습
    model = Lasso(alpha=alpha)
    model.fit(X_train_scaled, y_train)

    # 훈련 세트와 검증 세트에서의 예측 수행
    y_train_hat = model.predict(X_train_scaled)  
    y_val_hat = model.predict(X_val_scaled)  

    # R^2 값 계산
    train_r2 = r2_score(y_train, y_train_hat)
    val_r2 = r2_score(y_val, y_val_hat)
    
    # 결과 저장
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    # 결과 출력
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print('---------------------')

Alpha: 0.01
Train R^2: 0.7210
Validation R^2: 0.7309
---------------------
Alpha: 0.1
Train R^2: 0.7210
Validation R^2: 0.7317
---------------------
Alpha: 1
Train R^2: 0.7206
Validation R^2: 0.7373
---------------------
Alpha: 10
Train R^2: 0.7123
Validation R^2: 0.7632
---------------------
Alpha: 100
Train R^2: 0.4232
Validation R^2: 0.5380
---------------------


c:\Users\songs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.046e+08, tolerance: 1.582e+05
  model = cd_fast.enet_coordinate_descent(


In [132]:
model = Lasso(alpha=10) #10채택
model.fit(X_train_scaled, y_train)

Lasso(alpha=10)

In [133]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [134]:
coefficients = pd.DataFrame({
    'Feature': features,
    'Lasso Coefficient': model.coef_
})
print(coefficients)

         Feature  Lasso Coefficient
0           평균기온        1289.160279
1           최저기온         479.982498
2       일강수량(mm)          -0.000000
3     평균 풍속(m/s)          -0.000000
4     최소 상대습도(%)          -0.000000
5     평균 상대습도(%)          -0.000000
6   평균 현지기압(hPa)          -0.000000
7   평균 해면기압(hPa)          -0.000000
8    합계 일조시간(hr)          -0.000000
9   평균 전운량(1/10)           0.000000
10    1일전 모기 개체수        9248.540388
11    2일전 모기 개체수         992.408243
12    3일전 모기 개체수        2717.095055


In [135]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 956402.0288
Train RMSE : 977.9581
Train R^2 : 0.7123
Performance for TEST--------
Test MSE : 3134997.0751
Test RMSE : 1770.5923
Test R^2 : 0.4241
